In [1]:
import os
import numpy as np
import yaml
import cv2
import tqdm
import torch

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.object_detection import PyTorchFasterRCNN

In [2]:
#get the training test path, Number of classes and the classes from the config file
with open('/home/aya/Desktop/Kitti_FasterRCNN/data_configs/data.yaml') as file:
        data_configs = yaml.safe_load(file)

TEST_DIR_IMAGES = data_configs['TRAIN_DIR_IMAGES']
CLASSES = data_configs['CLASSES']
NUM_CLASSES = data_configs['NC']
SAVE_VALID_PREDICTIONS = data_configs['SAVE_VALID_PREDICTION_IMAGES']

DEST_DIR = "./adversarialExamples"
os.makedirs(DEST_DIR,exist_ok = True) #make the destination dir

In [3]:
# Create ART object detector
frcnn = PyTorchFasterRCNN(
        clip_values=(0, 255), attack_losses=["loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg"]
    )

/home/aya/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/aya/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/aya/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and will be removed in 0.15, please use 'weights_backbone' instead.
  warnings.warn(
/home/aya/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other tha

In [4]:
# Create and run attack
eps = 64
attack = ProjectedGradientDescent(estimator=frcnn, eps=eps, eps_step=2, max_iter=10)

In [5]:
for i in os.listdir(TEST_DIR_IMAGES):

    img = cv2.imread(os.path.join(TEST_DIR_IMAGES,i))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
    img = cv2.resize(img, dsize=(img.shape[1], img.shape[0]), interpolation=cv2.INTER_CUBIC)
    
    
    image = np.stack([img], axis=0).astype(np.float32)
    image_adv = attack.generate(x=image, y=None)

    cv2.imwrite(DEST_DIR+"/"+i, image_adv[0])

PGD - Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 10.91 GiB total capacity; 893.95 MiB already allocated; 51.81 MiB free; 934.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# # Stack images
# image = np.stack(image, axis=0).astype(np.float32)

In [ ]:
# print("number of images {}".format(len(filenames)))
# print("image.shape:", image.shape)

In [ ]:
# print("\nThe attack budget eps is {}".format(eps))
# print("The resulting maximal difference in pixel values is {}.".format(np.amax(np.abs(img - image_adv))))
